# Module Import Cell

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

## Data Import Cell

In [2]:
#Available population density by state and year meassured in persons per square mile
pop_df = pd.read_csv('data/Historical_Population_Density_States_only_RAND_US_1.csv')
#---------------------------------------------------------------------------------------
#Available firearm background check data
nics_df = pd.read_csv('data/nics-firearm-background-checks.csv')